In [1]:
import fire
import torch
from llama import Llama
from typing import List
from torchviz import make_dot

ckpt_dir = "/home/anyxling/llama2_weights/llama-2-7b/"
tokenizer_path = "/home/anyxling/llama2_weights/tokenizer.model"
temperature: float = 0.6
top_p: float = 0.9
max_seq_len: int = 128
max_gen_len: int = 64
max_batch_size: int = 2

In [2]:
import json
from pathlib import Path
from llama.model import ModelArgs, Transformer
from llama.tokenizer import Tokenizer
# generator = Llama.build(
#     ckpt_dir=ckpt_dir,
#     tokenizer_path=tokenizer_path,
#     max_seq_len=max_seq_len,
#     max_batch_size=max_batch_size,
# )
with open(Path(ckpt_dir) / "params.json", "r") as f:
    params = json.loads(f.read())
model_args: ModelArgs = ModelArgs(
    max_seq_len=max_seq_len,
    max_batch_size=max_batch_size,
    **params,
)
# tokenizer = Tokenizer(model_path=tokenizer_path)
model_args.vocab_size = 32000 # tokenizer.n_words
model_args.n_layers = 5 # limit the parameters due to limited memory
model = Transformer(model_args)
model.eval()
yhat = model(torch.randint(8, (2, 8), dtype=torch.long), 0)

In [3]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


### export to onnx

In [4]:
torch.onnx.export(model,
    (torch.randint(8, (2, 8), dtype=torch.long), 0), 
    f="/home/anyxling/llama2_weights/llama-2-7b/consolidated.00.onnx")

/home/anyxling/lxl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/anyxling/llama/llama/model.py:477: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if seqlen > 1:
